In [5]:
import numpy as np
import pandas as pd
from pprint import pprint

1. Cho mảng dữ liệu sau: AA=["co","co","co","khong","khong","khong","khong"]. Viết câu lệnh cho biết có bao nhiêu loại nhãn và số lần xuất hiện.

In [7]:
AA=np.array(["co","co","co","khong","khong","khong","khong"])
elements, count = np.unique(AA,return_counts=True)
print(elements,count)

['co' 'khong'] [3 4]


2. Viết câu lệnh tính Entropy cho mảng AA

In [8]:
# p0 = count[0]/len(AA) #len(AA) hoặc np.sum(count)
# p1=count[1]/len(AA)
# entropy = -p0*np.log2(p0)-p1*np.log2(p1)
# print(entropy)


In [9]:
entropy=np.sum([(-count[i]/np.sum(count))*np.log2(count[i]/np.sum(count)) for i in range(len(elements))])
entropy

0.9852281360342515

3. Viết hàm Entropy với đầu vào là một vector nhãn đầu ra là Entropy (theo công thức ở phần lý thuyết)

In [10]:
def Entropy(y):
    elements, count = np.unique(y,return_counts=True)
    E=np.sum([(-count[i]/np.sum(count))*np.log2(count[i]/np.sum(count)) for i in range(len(elements))])
    return E

4. Viết câu lệnh đọc dữ liệu từ file weather_tv.csv. In dữ liệu để kiểm tra

In [11]:
df = pd.read_csv(r'F:\study\HK1_3_2425\DS321_MachineLearning1\resources\weather1.csv',encoding='latin-1')
df

,outlook,temperature,humidity,wind,class
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rainy,mild,high,weak,yes
4,rainy,cool,normal,weak,yes
5,rainy,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rainy,mild,normal,weak,yes


6. Tách lấy cột nhãn của dữ liệu và tính Entropy của nó.

In [12]:
df.iloc[:,-1]

0      no
1      no
2     yes
3     yes
4     yes
5      no
6     yes
7      no
8     yes
9     yes
10    yes
11    yes
12    yes
13     no
Name: class, dtype: object

In [13]:
en=Entropy(df.iloc[:,-1])
en

0.9402859586706311

7. Viết câu lệnh lọc lấy dữ liệu với điều kiện Thời tiết là Nắng

In [14]:
sun = df.where(df['outlook']=='sunny').dropna()['class']
sun

0      no
1      no
7      no
8     yes
10    yes
Name: class, dtype: object

8. Viết câu lệnh để lọc dữ liệu với độ ẩm là Cao

In [15]:
hight=df[df['humidity']=='high']['class']
hight

0      no
1      no
2     yes
3     yes
7      no
11    yes
13     no
Name: class, dtype: object

In [16]:
en_hight=Entropy(hight)
en_hight

0.9852281360342515

9. Viết câu lệnh tạo mảng chứa các loại thông tin có trong cột thời tiết (Nắng, mưa, nhiều mây) và mảng lưu số lượng

In [17]:
outlook,count_o = np.unique(df['outlook'],return_counts=True)
print(outlook,count_o)

['overcast' 'rainy' 'sunny'] [4 5 5]


In [18]:
w=0
# Cách 1:
# for i in range(len(outlook)):
#   w+=count_o[i]/np.sum(count_o)*Entropy(df.where(df['outlook']==outlook[i]).dropna()['class'])
# Cách 2:
w=np.sum([count_o[i]/np.sum(count_o)*Entropy(df.where(df['outlook']==outlook[i]).dropna()['class']) for i in range(len(outlook))])
w

0.6935361388961918

10. Viết lệnh tính Gain cho cột thời tiết.

In [19]:
gain_ol=Entropy(df.iloc[:,-1])-w
gain_ol

0.24674981977443933

11. Viết hàm tính Gain.

In [20]:
def gain(data,split_attribute_name,target_name="class"):
  total_entropy=Entropy(data[target_name])
  vals,counts= np.unique(data[split_attribute_name],return_counts=True)
  for i in range(len(vals)):
    # sum+=counts[i]/np.sum(counts)*Entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name])
    sum=np.sum([counts[i]/np.sum(counts)*Entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
  information_gain = total_entropy-sum
  return information_gain
gain_c=gain(df,"class")
print(f"Gain(class) = {gain_c}")
gain_o=gain(df,"outlook")
print(f"Gain(outlook) = {gain_o}")

Gain(class) = 0.9402859586706311
Gain(outlook) = 0.24674981977443933


12. Viết câu lệnh lọc ra các cột có trong bảng thời tiết, trừ cột nhãn

In [21]:
features =df.columns[:-1]
features

Index(['outlook', 'temperature', 'humidity', 'wind'], dtype='object')

13. Viết câu lệnh tính Gain cho từng cột trong bảng, lưu vào một mảng

In [22]:
# result = {}
# for i in c_out:
#     x=gain(df,i)
#     result[i]=x
# result
item_values=[gain(df,feature) for feature in features]
item_values

[0.24674981977443933,
 0.02922256565895487,
 0.15183550136234159,
 0.04812703040826949]

14. Viết câu lệnh tình cột có Gain lớn nhất. Lưu vào cây.

In [23]:
# max_gain=max(result,key=result.get)
# decission_tree = {'node':max_gain,'next':{}}
# decission_tree
index = np.argmax(item_values)
best_feature = features[index]
best_feature

'outlook'

In [24]:
tree = {best_feature:{}}
tree

{'outlook': {}}

15. Loại cột được chọn ra khỏi mảng.

In [25]:
features=[i for i in features if i != best_feature]
features

['temperature', 'humidity', 'wind']

In [ ]:
def ID3(data,originaldata,features,target_attribute_name="class",parent_node_class = None):

    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]

    #If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    elif len(features) ==0:
        return parent_node_class
    else:
        #Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        #Select the feature which best splits the dataset
        item_values = [gain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]

        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        tree = {best_feature:{}}
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]

        #Grow a branch under the root node for each possible value of the root node feature

        for value in np.unique(data[best_feature]):
           # value = value
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = data.where(data[best_feature] == value).dropna()
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
            subtree = ID3(sub_data,df,features,target_attribute_name,parent_node_class)
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree
        return(tree)

ID3(df,df, df.columns[:-1])

{'outlook': {'overcast': 'yes',
  'rainy': {'wind': {'strong': 'no', 'weak': 'yes'}},
  'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}